In [1]:
# %cd ../..
# %rm -rf omega-modular-incl-eval/
# %pwd
# !git clone https://github.com/OndrejAlexaj/omega-modular-incl-eval.git
# %cd omega-modular-incl-eval/results
!pip install mizani
!pip install plotnine

In [2]:
import datetime
import pandas as pd
import re as re
import tabulate as tab
import math
from matplotlib.backends.backend_pdf import PdfPages
import matplotlib.pyplot as plt
# from IPython.display import display

import evallib as el

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [3]:
df_hy = el.read_file('hyperltl_exec.csv')
df_term = el.read_file('termination_exec.csv')
df_r = el.read_file('rabit_exec.csv')
df_combined = pd.concat([df_hy, df_term, df_r], ignore_index=True)

df_hy_summary = el.summary_stats(df_hy)
df_term_summary = el.summary_stats(df_term)
df_r_summary = el.summary_stats(df_r)
df_combined_summary = el.summary_stats(df_combined)

/home/xalexa09/incl_eval/.venv/lib/python3.10/site-packages/numpy/lib/_nanfunctions_impl.py:1215: RuntimeWarning: Mean of empty slice
/home/xalexa09/incl_eval/.venv/lib/python3.10/site-packages/numpy/lib/_nanfunctions_impl.py:1215: RuntimeWarning: Mean of empty slice
/home/xalexa09/incl_eval/.venv/lib/python3.10/site-packages/numpy/lib/_nanfunctions_impl.py:1215: RuntimeWarning: Mean of empty slice
/home/xalexa09/incl_eval/.venv/lib/python3.10/site-packages/numpy/lib/_nanfunctions_impl.py:1215: RuntimeWarning: Mean of empty slice
/home/xalexa09/incl_eval/.venv/lib/python3.10/site-packages/numpy/lib/_nanfunctions_impl.py:1215: RuntimeWarning: Mean of empty slice
/home/xalexa09/incl_eval/.venv/lib/python3.10/site-packages/numpy/lib/_nanfunctions_impl.py:1215: RuntimeWarning: Mean of empty slice
/home/xalexa09/incl_eval/.venv/lib/python3.10/site-packages/numpy/lib/_nanfunctions_impl.py:1215: RuntimeWarning: Mean of empty slice
/home/xalexa09/incl_eval/.venv/lib/python3.10/site-packages/nu

# **Correctness**

In [4]:
comparison = df_combined['kofola_fast-retcode'] == df_combined['autfilt_64-retcode']
filtered_comparison = comparison[~df_combined[['kofola_fast-retcode', 'autfilt_64-retcode']].isnull().any(axis=1)]
columns_are_equal = filtered_comparison.all()
columns_are_equal

np.True_

# **Execution time**

In [6]:
TIMEOUT = 900
TIMEOUT_VAL = TIMEOUT * 1.1
TIME_MIN = 0.001

# to_compare_exec_times = ["kofola_noopt_low-runtime", "bait-runtime", "forklift-runtime", "rabit-runtime"]
def plot_exec_t(df_eng, df_eng_summary, to_compare_exec_times, output_dir="plots"):
    import os

    os.makedirs(output_dir, exist_ok=True)

    df_eng_copy = df_eng.copy()
    df_eng_summary_copy = df_eng_summary.copy()

    # Generate unique pairs of methods
    pair_list = [(meth1, meth2) for meth1 in to_compare_exec_times for meth2 in to_compare_exec_times if meth1 != meth2]
    new_list = []
    for (meth1, meth2) in pair_list:
        if (meth2, meth1) not in new_list:
            new_list.append((meth1, meth2))
    pair_list = new_list

    MAX = 900000

    # Preprocess the data
    df_eng_copy = el.sanitize_results(df_eng_copy, df_eng_summary_copy, TIMEOUT_VAL)
    for eng in to_compare_exec_times:
        df_eng_copy[eng] = df_eng_copy[eng].apply(lambda x: x * 1000)

    # Generate and save plots
    for (meth1, meth2) in pair_list:
        plot = el.scatplot(df_eng_copy, {'x': meth1, 'y': meth2, 'max': MAX, 'states': False})
        
        file_name = f"{output_dir}/{meth1}_vs_{meth2}.pdf"
        plot.save(file_name, format='pdf', bbox_inches='tight')

        print(f"Plot saved as {file_name}")

    return

In [7]:
plot_exec_t(df_combined, df_combined_summary,["kofola_fast-runtime","autfilt_64-runtime"])
plot_exec_t(df_combined, df_combined_summary,["kofola_fast_early-runtime","autfilt_64-runtime"])
plot_exec_t(df_combined, df_combined_summary,["kofola_fast_early_plus-runtime","autfilt_64-runtime"])
plot_exec_t(df_combined, df_combined_summary,["kofola_fast_early_early_plus-runtime","autfilt_64-runtime"])

plot_exec_t(df_combined, df_combined_summary,["kofola_fast-runtime","rabit-runtime"])
plot_exec_t(df_combined, df_combined_summary,["kofola_fast-runtime","forklift-runtime"])
plot_exec_t(df_combined, df_combined_summary,["kofola_fast-runtime","bait-runtime"])

filtered_rows = df_combined_summary[
    df_combined_summary.index.str.endswith('-runtime') & ~df_combined_summary.index.str.contains('cnt')
]
filtered_rows

/home/xalexa09/incl_eval/.venv/lib/python3.10/site-packages/plotnine/ggplot.py:615: PlotnineWarning: Saving 8 x 8 in image.
/home/xalexa09/incl_eval/.venv/lib/python3.10/site-packages/plotnine/ggplot.py:616: PlotnineWarning: Filename: plots/kofola_fast-runtime_vs_autfilt_64-runtime.pdf


Plot saved as plots/kofola_fast-runtime_vs_autfilt_64-runtime.pdf


/home/xalexa09/incl_eval/.venv/lib/python3.10/site-packages/plotnine/ggplot.py:615: PlotnineWarning: Saving 8 x 8 in image.
/home/xalexa09/incl_eval/.venv/lib/python3.10/site-packages/plotnine/ggplot.py:616: PlotnineWarning: Filename: plots/kofola_fast_early-runtime_vs_autfilt_64-runtime.pdf


Plot saved as plots/kofola_fast_early-runtime_vs_autfilt_64-runtime.pdf


/home/xalexa09/incl_eval/.venv/lib/python3.10/site-packages/plotnine/ggplot.py:615: PlotnineWarning: Saving 8 x 8 in image.
/home/xalexa09/incl_eval/.venv/lib/python3.10/site-packages/plotnine/ggplot.py:616: PlotnineWarning: Filename: plots/kofola_fast_early_plus-runtime_vs_autfilt_64-runtime.pdf


...

/home/xalexa09/incl_eval/.venv/lib/python3.10/site-packages/plotnine/ggplot.py:615: PlotnineWarning: Saving 8 x 8 in image.
/home/xalexa09/incl_eval/.venv/lib/python3.10/site-packages/plotnine/ggplot.py:616: PlotnineWarning: Filename: plots/kofola_fast-runtime_vs_forklift-runtime.pdf


Plot saved as plots/kofola_fast-runtime_vs_forklift-runtime.pdf


/home/xalexa09/incl_eval/.venv/lib/python3.10/site-packages/plotnine/ggplot.py:615: PlotnineWarning: Saving 8 x 8 in image.
/home/xalexa09/incl_eval/.venv/lib/python3.10/site-packages/plotnine/ggplot.py:616: PlotnineWarning: Filename: plots/kofola_fast-runtime_vs_bait-runtime.pdf


Plot saved as plots/kofola_fast-runtime_vs_bait-runtime.pdf


,max,min,mean,median,std,timeouts
rabit-runtime,340.15,0.02,3.125732,0.08,19.727079,39.0
forklift-runtime,848.73,0.05,7.624224,0.09,59.403719,4.0
bait-runtime,872.37,0.06,9.588184,0.11,61.378891,25.0
kofola_fast-runtime,271.25,0.00,0.829857,0.00,11.469397,0.0
kofola_fast_early-runtime,270.87,0.00,1.041082,0.00,12.485873,0.0
kofola_fast_early_plus-runtime,270.02,0.00,1.035607,0.00,12.435759,0.0
kofola_fast_early_early_plus-runtime,272.91,0.00,1.061887,0.00,12.679144,0.0
autfilt_64-runtime,16.33,0.00,0.138054,0.01,0.920801,12.0


In [8]:
# comparing wins/loses
def compute_wins(df,method):
  all_methods = [
                 "autfilt_64",
                 "kofola_fast_early_early_plus",
                 "kofola_fast",
                 "bait",
                 "forklift",
                 "rabit"
                ]
  suffix = "-runtime"
  method_suf = method + suffix
  all_methods_suf = [m + suffix for m in all_methods]

  compare_methods = []
  for m in all_methods_suf:
    if (m != method_suf):
      compare_methods += [(method_suf, m)]

  dict_wins = {}
  for left, right in compare_methods:
        left_over_right = df[df[left] < df[right]]

        right_over_left = df[df[left] > df[right]]

        dict_wins[right] = {'wins': len(left_over_right),
                            'losses': len(right_over_left),
                           }
  return dict_wins

def print_win_table(df,method):
  dict_wins = compute_wins(df,method)
  tab_wins = []
  for key, val in dict_wins.items():
    tab_wins.append([key, val['wins'], val['losses']])
  headers_wins = ["method", "wins", "losses"]
  print(tab.tabulate(tab_wins, headers=headers_wins, tablefmt="github"))


df_sant = df_combined.copy()
df_sant = el.sanitize_results(df_sant, el.summary_stats(df_sant), TIMEOUT_VAL)
print_win_table(df_sant, "kofola_fast")
print("\n\n")

| method                               |   wins |   losses |
|--------------------------------------|--------|----------|
| autfilt_64-runtime                   |    514 |       48 |
| kofola_fast_early_early_plus-runtime |     81 |       93 |
| bait-runtime                         |    898 |        8 |
| forklift-runtime                     |    885 |       19 |
| rabit-runtime                        |    895 |       11 |





/home/xalexa09/incl_eval/.venv/lib/python3.10/site-packages/numpy/lib/_nanfunctions_impl.py:1215: RuntimeWarning: Mean of empty slice
/home/xalexa09/incl_eval/.venv/lib/python3.10/site-packages/numpy/lib/_nanfunctions_impl.py:1215: RuntimeWarning: Mean of empty slice
/home/xalexa09/incl_eval/.venv/lib/python3.10/site-packages/numpy/lib/_nanfunctions_impl.py:1215: RuntimeWarning: Mean of empty slice
/home/xalexa09/incl_eval/.venv/lib/python3.10/site-packages/numpy/lib/_nanfunctions_impl.py:1215: RuntimeWarning: Mean of empty slice
/home/xalexa09/incl_eval/.venv/lib/python3.10/site-packages/numpy/lib/_nanfunctions_impl.py:1215: RuntimeWarning: Mean of empty slice
/home/xalexa09/incl_eval/.venv/lib/python3.10/site-packages/numpy/lib/_nanfunctions_impl.py:1215: RuntimeWarning: Mean of empty slice
/home/xalexa09/incl_eval/.venv/lib/python3.10/site-packages/numpy/lib/_nanfunctions_impl.py:1215: RuntimeWarning: Mean of empty slice
/home/xalexa09/incl_eval/.venv/lib/python3.10/site-packages/nu

In [9]:
to_compare_states_cnt = [
                        "kofola_fast_cnt-states_cnt",
                        "kofola_fast_early_cnt-states_cnt",
                        "kofola_fast_early_plus_cnt-states_cnt",
                        "kofola_fast_early_early_plus_cnt-states_cnt",
                        ]
            
y_tool = "kofola_fast_cnt-states_cnt"
# y_tool = "autfilt_64_cnt-states_cnt"

def plot_states_cnt(df_eng, df_eng_summary, output_file="plots.pdf"):
  df_eng_copy = df_eng.copy()
  df_eng_summary_copy = df_eng_summary.copy()
  pair_list = [(meth1, y_tool) for meth1 in to_compare_states_cnt]

  MAX=df_eng_summary_copy['max'].max() * 1.8
  df_eng_copy = el.sanitize_results(df_eng_copy,df_eng_summary_copy,TIMEOUT_VAL)
  with PdfPages(output_file) as pdf:
    for (meth1, meth2) in pair_list:
      plot = el.scatplot(df_eng_copy, {'x': meth1, 'y': meth2, 'max': MAX, 'states': True})
      file_name = f"plots/{meth1}_vs_{meth2}.pdf"
      plot.save(file_name, format='pdf', bbox_inches='tight')

# **States_cnt**

In [10]:

plot_states_cnt(df_combined, df_combined_summary)

/home/xalexa09/incl_eval/.venv/lib/python3.10/site-packages/plotnine/ggplot.py:615: PlotnineWarning: Saving 8 x 8 in image.
/home/xalexa09/incl_eval/.venv/lib/python3.10/site-packages/plotnine/ggplot.py:616: PlotnineWarning: Filename: plots/kofola_fast_cnt-states_cnt_vs_autfilt_64_cnt-states_cnt.pdf
/home/xalexa09/incl_eval/.venv/lib/python3.10/site-packages/plotnine/ggplot.py:615: PlotnineWarning: Saving 8 x 8 in image.
/home/xalexa09/incl_eval/.venv/lib/python3.10/site-packages/plotnine/ggplot.py:616: PlotnineWarning: Filename: plots/kofola_fast_early_cnt-states_cnt_vs_autfilt_64_cnt-states_cnt.pdf
/home/xalexa09/incl_eval/.venv/lib/python3.10/site-packages/plotnine/ggplot.py:615: PlotnineWarning: Saving 8 x 8 in image.
/home/xalexa09/incl_eval/.venv/lib/python3.10/site-packages/plotnine/ggplot.py:616: PlotnineWarning: Filename: plots/kofola_fast_early_plus_cnt-states_cnt_vs_autfilt_64_cnt-states_cnt.pdf
/home/xalexa09/incl_eval/.venv/lib/python3.10/site-packages/plotnine/ggplot.py:6

In [42]:
filtered_rows = df_combined_summary[
    df_combined_summary.index.str.endswith('-states_cnt') & df_combined_summary.index.str.contains('_cnt-')
]# filtered_rows
filtered_rows

,max,min,mean,median,std,timeouts
kofola_fast_cnt-states_cnt,824363.0,2.0,2898.187638,13.0,41356.618642,0.0
kofola_fast_early_cnt-states_cnt,788073.0,2.0,2632.870861,13.0,38937.834893,0.0
kofola_fast_early_plus_cnt-states_cnt,788073.0,2.0,2632.471302,13.0,38937.843151,0.0
kofola_fast_early_early_plus_cnt-states_cnt,788073.0,2.0,2632.456954,13.0,38937.844103,0.0
autfilt_64_cnt-states_cnt,4810052.0,1.0,11709.420582,47.0,177302.852728,12.0


In [12]:
def compute_state_wins(df,method):
  all_methods = [
                 "kofola_fast_early_early_plus",
                 "kofola_fast"
                ]
  suffix = "_cnt-states_cnt"
  method_suf = method + suffix
  all_methods_suf = [m + suffix for m in all_methods]

  compare_methods = []
  for m in all_methods_suf:
    if (m != method_suf):
      compare_methods += [(method_suf, m)]

  dict_wins = {}
  for left, right in compare_methods:
        left_over_right = df[df[left] < df[right]]
        right_timeouts = left_over_right[left_over_right[right] == states_timeout]

        right_over_left = df[df[left] > df[right]]
        left_timeouts = right_over_left[right_over_left[left] == states_timeout]

        dict_wins[right] = {'wins': len(left_over_right),
                            'winsTO': len(right_timeouts),
                            'losses': len(right_over_left),
                            'lossesTO': len(left_timeouts),
                           }
  return dict_wins

def print_win_table_states_cnt(df, method):
  dict_wins = compute_state_wins(df, method)
  tab_wins = []
  for key, val in dict_wins.items():
    tab_wins.append([key, val['wins'], val['winsTO'], val['losses'], val['lossesTO']])
  headers_wins = ["method", "wins", "wins-timeouts", "losses", "losses-timeouts"]
  #table_to_file(tab_wins, headers_wins, out_prefix + "_table1right")
  print(tab.tabulate(tab_wins, headers=headers_wins, tablefmt="github"))


df_sant = df_combined.copy()
states_timeout = el.summary_stats(df_sant)['max'].max() * 1.8
df_sant = el.sanitize_results(df_sant, el.summary_stats(df_sant), TIMEOUT_VAL)
print_win_table_states_cnt(df_sant, "autfilt_64")
print("\n\n")

| method                                      |   wins |   wins-timeouts |   losses |   losses-timeouts |
|---------------------------------------------|--------|-----------------|----------|-------------------|
| kofola_fast_early_early_plus_cnt-states_cnt |     64 |               0 |      336 |                 2 |
| kofola_fast_cnt-states_cnt                  |     64 |               0 |      336 |                 2 |





/home/xalexa09/incl_eval/.venv/lib/python3.10/site-packages/numpy/lib/_nanfunctions_impl.py:1215: RuntimeWarning: Mean of empty slice
/home/xalexa09/incl_eval/.venv/lib/python3.10/site-packages/numpy/lib/_nanfunctions_impl.py:1215: RuntimeWarning: Mean of empty slice
/home/xalexa09/incl_eval/.venv/lib/python3.10/site-packages/numpy/lib/_nanfunctions_impl.py:1215: RuntimeWarning: Mean of empty slice
/home/xalexa09/incl_eval/.venv/lib/python3.10/site-packages/numpy/lib/_nanfunctions_impl.py:1215: RuntimeWarning: Mean of empty slice
/home/xalexa09/incl_eval/.venv/lib/python3.10/site-packages/numpy/lib/_nanfunctions_impl.py:1215: RuntimeWarning: Mean of empty slice
/home/xalexa09/incl_eval/.venv/lib/python3.10/site-packages/numpy/lib/_nanfunctions_impl.py:1215: RuntimeWarning: Mean of empty slice
